In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stockstats import StockDataFrame as Sdf
from ta import *
%matplotlib inline

nome_ficheiro = 'euro_new.txt'

try:
        fhand = open(nome_ficheiro)

except:
        print ('Nao e possivel abrir o ficheiro: ', fhand)


lista_quotes = []

#Cria lista com os ticket do SP500
for f in fhand:
    linha = f.strip().split(',')
    ticker = linha[0]
    lista_quotes.append(ticker)


for quote in lista_quotes:
    try:
        stock = pd.read_csv('dados_actuais_'+quote+'_050419.csv')
        output = stock
    except:
        continue    
    
    # Utilizando a livaria 'ta'
    # Add bollinger band high indicator filling NaN values
    output['bb_high_indicator'] = bollinger_hband_indicator(output["Close"], n=20, ndev=2, fillna=True)
    # Add bollinger band low indicator filling NaN values
    output['bb_low_indicator'] = bollinger_lband_indicator(output["Close"], n=20, ndev=2, fillna=True)
    # Adicionar Money Flow Index
    output['MoneyFlowIndex'] = money_flow_index(output["High"],output["Low"],output["Close"],output["Volume"],n=14, fillna=True)
    # Adicionar AwesomeOscillator
    output['AwesomeOscillator'] = ao(output["High"],output["Low"], s=5, l=34, fillna=True)
    # Adicionar Stochatic Oscillator
    output['StochaticOscillator'] = stoch(output["High"],output["Low"],output["Close"], n=14, fillna=True)
    # Adicionar Stochatic Oscillator Signal
    output['StochaticOscillatorSignal'] = stoch_signal(output["High"],output["Low"],output["Close"], n=14, d_n=3, fillna=True)
    # Adicionar True Strength Index
    output['TrueStrengthIndex'] = tsi(output["Close"], r=25, s=13, fillna=True)
    # Adicionar Ultimate Oscillator
    output['UltimateOscillator'] = uo(output["High"],output["Low"],output["Close"], s=7, m=14, l=28, ws=4.0, wm=2.0, wl=1.0, fillna=True)
    # Adicionar William % R
    output['William%R'] = wr(output["High"],output["Low"],output["Close"], lbp=14, fillna=True)
    # Adicionar RSI
    output['RSI'] = rsi(output["Close"], n=14, fillna=True)
    
    # Adicionar features (MM4,MM9,MM18) e uma label (1=Subiu, 0=Desceu)

    output['Change'] = output['Close'].pct_change()
    output['UP'] = np.where(output['Change'] >= 0.0,1,0)
    output['MM4'] = output['Close'].rolling(window=4).mean()
    output['MM9'] = output['Close'].rolling(window=9).mean()
    output['MM18'] = output['Close'].rolling(window=18).mean()
    output['CrossMM'] = np.where(output['MM4'] > output['MM18'],1.0,0.0)
    output['CrossPrice'] = np.where(output['Open'] > output['MM18'],1.0,0.0)
    
    # Preencher os dados em falta

    media4 = output['MM4'].mean()
    media9 = output['MM9'].mean()
    media18 = output['MM18'].mean()
    output['MM4'].fillna(media4, inplace=True)
    output['MM9'].fillna(media9, inplace=True)
    output['MM18'].fillna(media18, inplace=True)
    output['Change'].fillna(0, inplace=True)    
    
    
    # Criar tabela com 'features', e com respectiva label (UP)
    y = output['UP']
    X = output[['Volume','High','Low','RSI','CrossMM','CrossPrice','bb_low_indicator','bb_high_indicator','MoneyFlowIndex','Open','MM4','MM9','MM18','StochaticOscillator', 'William%R', 'UltimateOscillator', 'StochaticOscillatorSignal' ]]

    #Split 
    
    X_train = X.head(16)
    X_test  = X.tail(6)
    y_train = y.head(16)
    y_test = y.tail(6)
    
    #Scaling

    sc = StandardScaler()
    X_train_sca = sc.fit_transform(X_train)
    X_test_sca = sc.transform(X_test)
    
    #Abrir objecto que contém o modelo
    
    with open('meu_modelo','rb') as f:
        obj = pickle.load(f)
        
    
    # Apresenta resultados
        
    print(quote, '-->', obj.predict(X_test_sca))    
        
        


ADS.DE --> [0 1 1 1 1 1]
ALV.DE --> [0 0 1 1 0 0]
BAS.DE --> [0 1 1 1 1 1]
BAYN.DE --> [1 1 1 1 1 1]
BEI.DE --> [0 0 0 0 0 0]
BMW.DE --> [1 1 1 1 1 1]
CBK.DE --> [0 1 1 0 0 1]
CON.DE --> [1 1 1 1 0 1]
DAI.DE --> [1 1 1 0 1 0]
DB1.DE --> [0 0 0 0 0 0]
DBK.DE --> [0 1 0 0 0 0]
DPW.DE --> [0 1 0 1 1 0]
DTE.DE --> [0 0 0 0 1 0]
EOAN.DE --> [0 0 1 0 0 0]
FME.DE --> [0 0 0 0 0 0]
FRE.DE --> [1 1 0 1 0 0]
HEI.DE --> [1 1 1 1 0 1]
HEN3.DE --> [1 0 0 0 1 0]
IFX.DE --> [1 1 1 1 1 1]
LHA.DE --> [0 0 1 1 1 1]
LIN.DE --> [1 1 1 1 1 1]
MRK.DE --> [0 0 0 0 0 1]
MUV2.DE --> [0 0 1 0 1 0]
PSM.DE --> [1 1 1 1 1 1]
RWE.DE --> [0 0 0 0 0 0]
SAP.DE --> [1 0 0 0 0 0]
SIE.DE --> [0 1 1 0 0 1]
TKA.DE --> [1 1 1 1 0 0]
VNA.DE --> [0 0 0 0 0 0]
VOW3.DE --> [1 1 1 1 1 0]
AC.PA --> [1 1 1 1 0 0]
ACA.PA --> [0 1 1 1 0 0]
AI.PA --> [0 1 0 0 0 0]
AIR.PA --> [1 0 0 0 0 0]
BN.PA --> [1 0 0 0 0 0]
BNP.PA --> [0 1 1 1 1 0]
CA.PA --> [1 0 0 0 0 0]
CAP.PA --> [1 1 0 1 1 1]
CS.PA --> [1 1 1 1 0 0]
DG.PA --> [1 0 0 0 0 0]
E